In [1]:
import sqlalchemy as sa
import sqlalchemy.ext.declarative as sad

In [2]:
engine = sa.create_engine('sqlite:///data/nobel_prize.db', echo=True)

In [3]:
Base = sad.declarative_base()

In [4]:
class Winner(Base):
    __tablename__ = 'winners'
    
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String)
    category = sa.Column(sa.String)
    year = sa.Column(sa.Integer)
    nationality = sa.Column(sa.String)
    sex = sa.Column(sa.Enum('male','female'))
    
    def __repr__(self):
        return "<Winner(name='%s', category='%s', year='%s')"%(self.name, self.category, self.year)

In [6]:
Base.metadata.create_all(engine)

2016-12-20 14:43:39,388 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-20 14:43:39,389 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 14:43:39,391 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-20 14:43:39,392 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 14:43:39,394 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2016-12-20 14:43:39,395 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 14:43:39,397 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winners (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	category VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2016-12-20 14:43:39,398 INFO sqlalchemy.engine.base.Engine ()
2016-12-20 14:43:39,401 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
import sqlalchemy.orm as sorm

In [8]:
Session = sorm.sessionmaker(bind=engine)
session = Session()

In [9]:
data = [{'category': 'Physics',
         'name': 'Albert Einstein',
         'nationality': 'Swiss',
         'sex': 'male',
         'year': 1921},
        {'category': 'Physics',
         'name': 'Paul Dirac',
         'nationality': 'British',
         'sex': 'male',
         'year': 1933},
        {'category': 'Chemistry',
         'name': 'Marie Curie',
         'nationality': 'Polish',
         'sex': 'female',
         'year': 1911}]
data

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [10]:
albert = Winner(**data[0])
session.add(albert)
session.new

IdentitySet([<Winner(name='Albert Einstein', category='Physics', year='1921')])

In [11]:
session.expunge(albert)
session.new

IdentitySet([])

In [12]:
winner_rows = [Winner(**w) for w in data]
session.add_all(winner_rows)
session.commit()

2016-12-20 14:43:49,853 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 14:43:49,855 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 14:43:49,858 INFO sqlalchemy.engine.base.Engine ('Albert Einstein', 'Physics', 1921, 'Swiss', 'male')
2016-12-20 14:43:49,862 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 14:43:49,863 INFO sqlalchemy.engine.base.Engine ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2016-12-20 14:43:49,865 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2016-12-20 14:43:49,866 INFO sqlalchemy.engine.base.Engine ('Marie Curie', 'Chemistry', 1911, 'Polish', 'female')
2016-12-20 14:43:49,867 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
session.query(Winner).count()

2016-12-20 14:43:49,879 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 14:43:49,881 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners) AS anon_1
2016-12-20 14:43:49,883 INFO sqlalchemy.engine.base.Engine ()


3

In [14]:
result = session.query(Winner).filter_by(nationality = 'Swiss')

In [15]:
list(result)

2016-12-20 14:43:49,902 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2016-12-20 14:43:49,904 INFO sqlalchemy.engine.base.Engine ('Swiss',)


[<Winner(name='Albert Einstein', category='Physics', year='1921')]

In [16]:
list(session.query(Winner).filter(Winner.nationality == 'Swiss'))

2016-12-20 14:43:49,919 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2016-12-20 14:43:49,920 INFO sqlalchemy.engine.base.Engine ('Swiss',)


[<Winner(name='Albert Einstein', category='Physics', year='1921')]

In [17]:
result = session.query(Winner).filter(Winner.category == 'Physics',
                                      Winner.category != 'Swiss')
list(result)

2016-12-20 14:43:49,930 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.category = ? AND winners.category != ?
2016-12-20 14:43:49,932 INFO sqlalchemy.engine.base.Engine ('Physics', 'Swiss')


[<Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933')]

In [18]:
session.query(Winner).get(3)

2016-12-20 14:43:49,942 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2016-12-20 14:43:49,944 INFO sqlalchemy.engine.base.Engine (3,)


<Winner(name='Marie Curie', category='Chemistry', year='1911')

In [19]:
result = session.query(Winner).order_by('year')
list(result)

2016-12-20 14:43:49,952 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners ORDER BY winners.year
2016-12-20 14:43:49,954 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Marie Curie', category='Chemistry', year='1911'),
 <Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933')]

In [20]:
def sa_inst_to_dict(inst, remove_id=True):
    d = {}
    
    for column in inst.__table__.columns:
        d[column.name] = getattr(inst, column.name)
    
    if remove_id:
        d.pop('id')

    return d

In [21]:
winner_rows = session.query(Winner)
nobel_winners = [sa_inst_to_dict(w) for w in winner_rows]
nobel_winners

2016-12-20 14:43:49,974 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2016-12-20 14:43:49,976 INFO sqlalchemy.engine.base.Engine ()


[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [22]:
marie = session.query(Winner).get(3)
marie.nationality = 'French'
session.dirty

IdentitySet([<Winner(name='Marie Curie', category='Chemistry', year='1911')])

In [23]:
session.commit()

2016-12-20 14:43:49,993 INFO sqlalchemy.engine.base.Engine UPDATE winners SET nationality=? WHERE winners.id = ?
2016-12-20 14:43:49,995 INFO sqlalchemy.engine.base.Engine ('French', 3)
2016-12-20 14:43:49,997 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
session.dirty

IdentitySet([])

In [25]:
session.query(Winner).get(3).nationality

2016-12-20 14:43:50,014 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-12-20 14:43:50,016 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2016-12-20 14:43:50,017 INFO sqlalchemy.engine.base.Engine (3,)


'French'

In [26]:
#session.query(Winner).filter_by(name='Albert Einstein').delete()

In [27]:
list(session.query(Winner))

2016-12-20 14:43:50,032 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2016-12-20 14:43:50,034 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name='Albert Einstein', category='Physics', year='1921'),
 <Winner(name='Paul Dirac', category='Physics', year='1933'),
 <Winner(name='Marie Curie', category='Chemistry', year='1911')]

In [28]:
#Winner.__table__.drop(engine)

# Mongo experimentation

I installed Mongo via Homebrew, and then I used 'brew services start mongodb', I believe to start it at login. I can run 'brew services' to see how to stop this (and keep it from starting).

In [29]:
import pymongo

In [33]:
DB_NOBEL_PRIZE = 'nobel_prize'
COLL_WINNERS = 'winners'

In [36]:
def get_mongo_database(db_name, host='localhost', 
                       port=27017, username=None, password=None):
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s'%(username, password, host, db_name)
        conn = pymongo.MongoClient(mongo_uri)
    else:
        conn = pymongo.MongoClient(host, port)
        
    return conn[db_name]

In [37]:
db = get_mongo_database(DB_NOBEL_PRIZE)
coll = db[COLL_WINNERS]

In [48]:
coll.insert_many(nobel_winners)

BulkWriteError: batch op errors occurred

In [47]:
nobel_winners

[{'_id': ObjectId('5859a7d65c0a213b957c23b2'),
  'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'_id': ObjectId('5859a7d65c0a213b957c23b3'),
  'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'_id': ObjectId('5859a7d65c0a213b957c23b4'),
  'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [50]:
list(coll.find({'category':'Chemistry'}))

[{'_id': ObjectId('5859a7d65c0a213b957c23b4'),
  'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [51]:
res = coll.find({'year': {'$gt': 1930}})
list(res)

[{'_id': ObjectId('5859a7d65c0a213b957c23b3'),
  'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933}]

In [54]:
res = coll.find({'$or':[{'year': {'$gt': 1930}}, {'sex':'female'}]})
list(res)

[{'_id': ObjectId('5859a7d65c0a213b957c23b3'),
  'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'_id': ObjectId('5859a7d65c0a213b957c23b4'),
  'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [55]:
def mongo_coll_to_dicts(dbname='test', collname='test',
                        query={}, del_id=True, **kw):
    db = get_mongo_database(dbname, **kw)
    res = list(db[collname].find(query))
    
    if del_id:
        for r in res:
            r.pop('_id')
            
    return res

In [56]:
mongo_coll_to_dicts(DB_NOBEL_PRIZE, COLL_WINNERS)

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]